In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset

In [2]:
import matplotlib.pyplot as plt

In [3]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

# Select variable

In [11]:
#Variable 
var="tg_mean"

files85 = glob.glob(folder+"*"+"rcp85_"+var+"_annual.nc")
files45 = glob.glob(folder+"*"+"rcp45_"+var+"_annual.nc")
filesobs = glob.glob(folder+"NRCAN_obs_"+var+"_annual.nc")

#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45


['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MPI-ESM-LR_rcp45_tg_mean_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NorESM1-M_rcp45_tg_mean_annual.nc']

In [12]:
filesobs

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NRCAN_obs_tg_mean_annual.nc']

# FOR RCP45 

In [6]:
dsEns45= ens.create_ensemble(files45)
#dsEnsSeas45 = ens.create_ensemble(filesSeas)
dsEns45


<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 151)
Coordinates:
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * time         (time) datetime64[ns] 1950-01-01 1951-01-01 ... 2100-01-01
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    tg_mean      (realization, time, lat, lon) float32 dask.array<chunksize=(1, 10, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.

## Get the percentiles

In [7]:
# Calculate ensembles percentiles on regional means
perctiles45 = ens.ensemble_percentiles(dsEns45)
perctiles45
dsEns45 =0

## Convert in Dataframe

In [8]:
dfper145 = perctiles45.to_dataframe().dropna()
dfper45 = perctiles45.drop('realization').to_dataframe().dropna()
dfmean45 = dfper45.reset_index()
dfper45 = 0

In [9]:
dfmean45

,lat,lon,time,tg_mean_p10,tg_mean_p50,tg_mean_p90
0,66.623306,-89.045212,1950-01-01,257.085693,258.653290,260.071655
1,66.623306,-89.045212,1951-01-01,256.835785,258.389740,259.078644
2,66.623306,-89.045212,1952-01-01,257.146881,258.677917,259.168579
3,66.623306,-89.045212,1953-01-01,257.813049,258.613068,259.164734
4,66.623306,-89.045212,1954-01-01,258.522583,259.148529,259.997162
...,...,...,...,...,...,...
11507101,40.041039,-74.045807,2096-01-01,287.267639,287.622437,289.489594
11507102,40.041039,-74.045807,2097-01-01,286.741150,287.997528,288.674133
11507103,40.041039,-74.045807,2098-01-01,286.326630,287.628967,288.842590
11507104,40.041039,-74.045807,2099-01-01,286.937592,287.839813,290.203461


In [13]:
dfmean45.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp45"+var+"_forTS.feather")

In [ ]:
dfmean45 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp45"+var+"_forTS.feather")

# FOR RCP85 

In [14]:
dsEns85= ens.create_ensemble(files85)
dsEns85


<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 151)
Coordinates:
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * time         (time) datetime64[ns] 1950-01-01 1951-01-01 ... 2100-01-01
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    tg_mean      (realization, time, lat, lon) float32 dask.array<chunksize=(1, 10, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.

## Get the percentiles

In [15]:
# Calculate ensembles percentiles on regional means
perctiles85 = ens.ensemble_percentiles(dsEns85)
perctiles85
dsEns85 =0

## Convert in Dataframe

In [16]:
dfper185 = perctiles85.to_dataframe().dropna()
dfper85 = perctiles85.drop('realization').to_dataframe().dropna()
dfmean85 = dfper85.reset_index()
dfper85 = 0

In [17]:
dfmean85

,lat,lon,time,tg_mean_p10,tg_mean_p50,tg_mean_p90
0,66.623306,-89.045212,1950-01-01,257.056732,258.643677,260.095032
1,66.623306,-89.045212,1951-01-01,256.838013,258.383789,259.087128
2,66.623306,-89.045212,1952-01-01,257.158539,258.634918,259.196228
3,66.623306,-89.045212,1953-01-01,257.833679,258.571075,259.167938
4,66.623306,-89.045212,1954-01-01,258.513580,259.165405,260.001831
...,...,...,...,...,...,...
11507101,40.041039,-74.045807,2096-01-01,289.641998,290.565857,292.025970
11507102,40.041039,-74.045807,2097-01-01,289.080017,290.811249,291.779266
11507103,40.041039,-74.045807,2098-01-01,288.290466,290.388062,291.735809
11507104,40.041039,-74.045807,2099-01-01,289.418976,290.153625,291.091431


In [18]:
dfmean85.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp85"+var+"_forTS.feather")

In [ ]:
dfmean85 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp85"+var+"_forTS.feather")

# FOR OBSERVED DATA

In [19]:
dsOBs= xr.open_mfdataset(filesobs)

will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  dsOBs= xr.open_mfdataset(filesobs)

to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  combined = auto_combine(



In [20]:
dsOBs

<xarray.Dataset>
Dimensions:  (lat: 510, lon: 1068, time: 64)
Coordinates:
  * time     (time) datetime64[ns] 1950-01-01 1951-01-01 ... 2013-01-01
  * lon      (lon) float32 -140.95833 -140.875 ... -52.125004 -52.04167
  * lat      (lat) float32 83.45833 83.375 83.291664 ... 41.125 41.041668
Data variables:
    tg_mean  (time, lat, lon) float32 dask.array<chunksize=(64, 510, 1068), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           NRCAN 10km Gridded Climate Dataset
    history:         2012-10-22T13:14:41: Convert from original format to Net...
    institution:     NRCAN
    source:          ANUSPLIN
    redistribution:  Redistribution policy unknown. For internal use only.

## Convert in Dataframe

In [21]:
dfperObs = dsOBs.to_dataframe().dropna()
dfmeanObs = dfperObs.reset_index()
dfperObs = 0

In [22]:
dfmeanObs

,lat,lon,time,tg_mean
0,83.125000,-74.541672,1950-01-01,251.429901
1,83.125000,-74.541672,1951-01-01,252.994797
2,83.125000,-74.541672,1952-01-01,253.964935
3,83.125000,-74.541672,1953-01-01,253.616013
4,83.125000,-74.541672,1954-01-01,253.628616
...,...,...,...,...
16464827,41.708332,-82.208336,2009-01-01,282.584686
16464828,41.708332,-82.208336,2010-01-01,283.870300
16464829,41.708332,-82.208336,2011-01-01,284.144257
16464830,41.708332,-82.208336,2012-01-01,285.570129


In [23]:
dfmeanObs.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcpObs"+var+"_forTS.feather")

In [ ]:
dfmeanObs = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcpObs"+var+"_forTS.feather")

## Merge all 3 dataframes

In [44]:
df2m =  pd.merge(dfmean45, dfmean85, on=["lat","lon", "time"], suffixes=('_45', '_85'))

In [45]:
df2m

,lat,lon,time,tg_mean_p10_45,tg_mean_p50_45,tg_mean_p90_45,tg_mean_p10_85,tg_mean_p50_85,tg_mean_p90_85
0,66.623306,-89.045212,1950-01-01,257.085693,258.653290,260.071655,257.056732,258.643677,260.095032
1,66.623306,-89.045212,1951-01-01,256.835785,258.389740,259.078644,256.838013,258.383789,259.087128
2,66.623306,-89.045212,1952-01-01,257.146881,258.677917,259.168579,257.158539,258.634918,259.196228
3,66.623306,-89.045212,1953-01-01,257.813049,258.613068,259.164734,257.833679,258.571075,259.167938
4,66.623306,-89.045212,1954-01-01,258.522583,259.148529,259.997162,258.513580,259.165405,260.001831
...,...,...,...,...,...,...,...,...,...
11507101,40.041039,-74.045807,2096-01-01,287.267639,287.622437,289.489594,289.641998,290.565857,292.025970
11507102,40.041039,-74.045807,2097-01-01,286.741150,287.997528,288.674133,289.080017,290.811249,291.779266
11507103,40.041039,-74.045807,2098-01-01,286.326630,287.628967,288.842590,288.290466,290.388062,291.735809
11507104,40.041039,-74.045807,2099-01-01,286.937592,287.839813,290.203461,289.418976,290.153625,291.091431


In [46]:
df3m =  pd.merge(df2m, dfmeanObs, on=["lat","lon", "time"])

In [47]:
df3m

,lat,lon,time,tg_mean_p10_45,tg_mean_p50_45,tg_mean_p90_45,tg_mean_p10_85,tg_mean_p50_85,tg_mean_p90_85,tg_mean


In [ ]:
df3m = df3m.rename(columns={'oldName1': 'tg_mean_Obs', 'oldName2': 'newName2'})

## Cut by region

## Get the lat lon regions

In [33]:
regions_folder = "/home/mlopez/EXEC/Grids-polygons-regions/"

In [39]:
files_regions = glob.glob(regions_folder+"*"+"lat_lon.feather")
files_regions

['/home/mlopez/EXEC/Grids-polygons-regions/DDE_STF_20K_REG_FOR_VUE_Slat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/DDE_STF_20K_UA_PER_VUE_Slat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/DOM_BIOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/REG_ECOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/SDOM_BIOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/Secteurs_Operations_Regionaleslat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/SREG_ECOlat_lon.feather',
 '/home/mlopez/EXEC/Grids-polygons-regions/territoire_guidelat_lon.feather']

In [40]:
listdfreg=[]
for region in files_regions:
    df = pd.read_feather(region)
    listdfreg.append(df)
   

In [41]:
listdfreg[1]

,index,lat,lon,index_right,OBJECTID,PER_NO_UA,DATE_PUBLI,SUPERFICIE,Shape_Leng,Shape_Area,D_GENERAl
0,71387,52.373878,-68.129379,40,41,09352,2019-11-30,1424519.6,1.241968e+06,1.404552e+10,Nord-Est
1,71803,52.290550,-68.129379,40,41,09352,2019-11-30,1424519.6,1.241968e+06,1.404552e+10,Nord-Est
2,71804,52.290550,-68.046051,40,41,09352,2019-11-30,1424519.6,1.241968e+06,1.404552e+10,Nord-Est
3,71805,52.290550,-67.962723,40,41,09352,2019-11-30,1424519.6,1.241968e+06,1.404552e+10,Nord-Est
4,71806,52.290550,-67.879387,40,41,09352,2019-11-30,1424519.6,1.241968e+06,1.404552e+10,Nord-Est
...,...,...,...,...,...,...,...,...,...,...,...
15220,487851,45.458332,-70.708336,69,70,05151,2019-11-30,9572.5,5.764414e+04,9.594909e+07,None
15221,106297,45.374161,-70.962601,2,3,05151,2019-11-30,1657.5,3.229192e+04,1.662015e+07,None
15222,106711,45.290829,-71.129257,5,6,05151,2019-11-30,2353.5,3.875096e+04,2.360611e+07,None
15223,489982,45.291668,-71.125000,5,6,05151,2019-11-30,2353.5,3.875096e+04,2.360611e+07,None


In [42]:
lisdfregdataObs =[]
for dfregion in listdfreg:
    dfdataregObs = pd.merge(dfregion, dfmeanObs, on=["lat","lon"])
    lisdfregdataObs.append(dfdataregObs)

In [43]:
lisdfregdataObs[1]

,index,lat,lon,index_right,OBJECTID,PER_NO_UA,DATE_PUBLI,SUPERFICIE,Shape_Leng,Shape_Area,D_GENERAl,time,tg_mean
0,399238,52.375,-68.125000,40,41,09352,2019-11-30,1424519.6,1.241968e+06,1.404552e+10,Nord-Est,1950-01-01,268.889191
1,399238,52.375,-68.125000,40,41,09352,2019-11-30,1424519.6,1.241968e+06,1.404552e+10,Nord-Est,1951-01-01,270.131348
2,399238,52.375,-68.125000,40,41,09352,2019-11-30,1424519.6,1.241968e+06,1.404552e+10,Nord-Est,1952-01-01,270.910583
3,399238,52.375,-68.125000,40,41,09352,2019-11-30,1424519.6,1.241968e+06,1.404552e+10,Nord-Est,1953-01-01,270.765411
4,399238,52.375,-68.125000,40,41,09352,2019-11-30,1424519.6,1.241968e+06,1.404552e+10,Nord-Est,1954-01-01,269.805450
...,...,...,...,...,...,...,...,...,...,...,...,...,...
486971,482465,45.875,-74.541672,151,152,06151,2019-11-30,2251.4,2.715512e+04,2.252438e+07,None,2009-01-01,277.594421
486972,482465,45.875,-74.541672,151,152,06151,2019-11-30,2251.4,2.715512e+04,2.252438e+07,None,2010-01-01,279.617371
486973,482465,45.875,-74.541672,151,152,06151,2019-11-30,2251.4,2.715512e+04,2.252438e+07,None,2011-01-01,278.916229
486974,482465,45.875,-74.541672,151,152,06151,2019-11-30,2251.4,2.715512e+04,2.252438e+07,None,2012-01-01,279.216827


In [30]:
lisdfregdata45 =[]
for dfregion in listdfreg:
    dfdatareg45 = pd.merge(dfregion, dfmean45, on=["lat","lon"])
    lisdfregdata45.append(dfdatareg45)

In [ ]:
listdfreg=[]

In [31]:
dfmeanUA45 = lisdfregdata45[1]
dfmeanUA45

,index,lat,lon,index_right,OBJECTID,PER_NO_UA,DATE_PUBLI,SUPERFICIE,Shape_Leng,Shape_Area,D_GENERAl,time,tg_mean_p10,tg_mean_p50,tg_mean_p90
0,71387,52.373878,-68.129379,40,41,09352,2019-11-30,1424519.6,1.241968e+06,1.404552e+10,Nord-Est,1950-01-01,268.789032,269.166229,270.688904
1,71387,52.373878,-68.129379,40,41,09352,2019-11-30,1424519.6,1.241968e+06,1.404552e+10,Nord-Est,1951-01-01,267.825256,268.893188,269.469147
2,71387,52.373878,-68.129379,40,41,09352,2019-11-30,1424519.6,1.241968e+06,1.404552e+10,Nord-Est,1952-01-01,268.255157,269.260284,270.207703
3,71387,52.373878,-68.129379,40,41,09352,2019-11-30,1424519.6,1.241968e+06,1.404552e+10,Nord-Est,1953-01-01,268.328583,269.103821,270.563354
4,71387,52.373878,-68.129379,40,41,09352,2019-11-30,1424519.6,1.241968e+06,1.404552e+10,Nord-Est,1954-01-01,268.174011,269.902985,270.895325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150011,106711,45.290829,-71.129257,5,6,05151,2019-11-30,2353.5,3.875096e+04,2.360611e+07,None,2096-01-01,278.609772,280.226044,281.615631
1150012,106711,45.290829,-71.129257,5,6,05151,2019-11-30,2353.5,3.875096e+04,2.360611e+07,None,2097-01-01,278.376892,280.244904,281.501831
1150013,106711,45.290829,-71.129257,5,6,05151,2019-11-30,2353.5,3.875096e+04,2.360611e+07,None,2098-01-01,278.167236,280.005920,280.946625
1150014,106711,45.290829,-71.129257,5,6,05151,2019-11-30,2353.5,3.875096e+04,2.360611e+07,None,2099-01-01,278.631653,280.080017,282.050476


## Get mean by year and subregion

In [ ]:
dfmean45UA = dfmeanUA45.groupby(["time", "PER_NO_UA"]).mean()

In [ ]:
dfmean45UA2 = dfmean45UA.reset_index()

In [ ]:
dfmean45UA2

## Split by subregion

In [ ]:
dfmean45UA3 = dfmean45UA2[['time', 'PER_NO_UA', 'tg_mean_p10', 'tg_mean_p50', 'tg_mean_p90']]

In [ ]:
dfmean45UA3.PER_NO_UA.unique()

In [ ]:
listdfUA = []
for sr in dfmean45UA3.PER_NO_UA.unique():
    dfUAsr = dfmean45UA3.loc[dfmean45UA3['PER_NO_UA'] == sr].copy()
    listdfUA.append(dfUAsr)
    

In [ ]:
dfUA451 = listdfUA[1]

## 4th degre polynomium

In [ ]:
def polysmooth(years, y, degree):
    coef = np.polyfit(years, y, degree)
    return np.polyval(coef,years)

In [ ]:
years = dfUA451.time.dt.year

In [ ]:
rp1 = polysmooth(years,dfUA451.tg_mean_p10,4)
rp5 = polysmooth(years,dfUA451.tg_mean_p50,4)
rp9 = polysmooth(years,dfUA451.tg_mean_p90,4)

## Create DF to plot TimeSeries graph in R

In [ ]:
df45 = pd.DataFrame()
df45["time"] = years.values
dfUA451["tg_mean_p10"] = rp1
dfUA451["tg_mean_p50"] = rp5
dfUA451["tg_mean_p90"] = rp9
dfUA451["tg_mean_p10"] = round(dfUA451["tg_mean_p10"]-273.15, 2)
dfUA451["tg_mean_p50"] = round(dfUA451["tg_mean_p50"]-273.15, 2)
dfUA451["tg_mean_p90"] = round(dfUA451["tg_mean_p90"]-273.15, 2)

In [ ]:
dfUA451

In [ ]:
#plt.plot(years, dfUA451["tg_mean_p50"], markersize='10', color='blue')
#plt.fill_between(years, polysmooth(years, dfUA451.tg_mean_p10,4), polysmooth(years, dfUA451.tg_mean_p90,4), alpha=0.3, color='blue')

## TO DO: 
- Do same for all regions - loop
- Do same for RCP 85 and observed data
- Merge all 3 results for every region to get one csv file 

## Create DF by region

In [ ]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
            "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"),  
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

In [ ]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [ ]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

In [32]:
dfmean45

,lat,lon,time,tg_mean_p10,tg_mean_p50,tg_mean_p90
0,66.623306,-89.045212,1950-01-01,257.085693,258.653290,260.071655
1,66.623306,-89.045212,1951-01-01,256.835785,258.389740,259.078644
2,66.623306,-89.045212,1952-01-01,257.146881,258.677917,259.168579
3,66.623306,-89.045212,1953-01-01,257.813049,258.613068,259.164734
4,66.623306,-89.045212,1954-01-01,258.522583,259.148529,259.997162
...,...,...,...,...,...,...
11507101,40.041039,-74.045807,2096-01-01,287.267639,287.622437,289.489594
11507102,40.041039,-74.045807,2097-01-01,286.741150,287.997528,288.674133
11507103,40.041039,-74.045807,2098-01-01,286.326630,287.628967,288.842590
11507104,40.041039,-74.045807,2099-01-01,286.937592,287.839813,290.203461


In [ ]:
for region, (name, short) in short_dict.items():
    print(region)
    print ("Cutting by region")
    for region1 in files_regions:
        dfregion = pd.read_feather(region1)
        print ("Merging region with data")
        dfdatareg45 = pd.merge(dfregion, dfmean45, on=["lat","lon"])
        print ("Get mean by year for every sub-region")
        dfmeanUA45 = dfdatareg45
        dfmean45UA = dfmeanUA45.groupby(["time", name]).mean()
        dfmean45UA2 = dfmean45UA.reset_index()
        print ("Split by subregion")
        dfmean45UA3 = dfmean45UA2[['time', name, 'tg_mean_p10', 'tg_mean_p50', 'tg_mean_p90']]
        for name,sr in dfmean45UA3.groupby("name"):
            dfUAsr = dfmean45UA3.loc[dfmean45UA3[name] == sr2]
            dfUA451 = dfUAsr.copy()
            print ("4th Polynomium")
            years = dfUA451.time.dt.year
            for p in ('_p10', '_p50', '_p90'):
                dfUA451["tg_mean"+p] = round(polysmooth(years,dfUA451.variable+p,4) - 273.15, 2)
            rp5 = polysmooth(years,dfUA451.variable+"_p10",4)
            rp9 = polysmooth(years,dfUA451.variable+"_p10",4)
            print ("Create DF to plot TimeSeries graph in R")
            df45 = pd.DataFrame()
            df45["time"] = years.values
            dfUA451["tg_mean_p10"] = rp1
            dfUA451["tg_mean_p50"] = rp5
            dfUA451["tg_mean_p90"] = rp9
            dfUA451["tg_mean_p10"] = round(dfUA451[variable+"_p10"]-273.15, 2)
            dfUA451["tg_mean_p50"] = round(dfUA451variable+["_p50"]-273.15, 2)
            dfUA451["tg_mean_p90"] = round(dfUA451[variable+"_p90"]-273.15, 2)
            lfinaldf45.append(dfUA451)
        
        TO-DO: 
            - Same for rcp85
            - Same for observed data?
            - merge all 3 df
            print to csv
        
    
    
    
    
    
    
    dfpolyshape = from_geofeather('/home/mlopez/EXEC/Grids-polygons-regions/Grid-'+region+'.feather')
    dftp = pd.merge(dftgseall3, dfpolyshape, on=["lat","lon"])
    print ("Merged with polygons")
    listTG = []
    for tg in dftp[name].unique().tolist():
        df2 = dftp[dftp[name] == tg]
        print (tg)
        if tg != None:
            listTG.append(df2)
        #print (listTG)
    for df in listTG:
        geometry = df["geometry"]
        crs = {'init': "epsg:4326"}
        gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
        print (gdf[name].iloc[0])
        #Substitute filename accents
        gdf.to_file(replace_all("/home/mlopez/EXEC/GeoJsonMFFP/"+gdf[name].iloc[0], d)+"_"+var+"_monthly.json", driver="GeoJSON")
    geometry = dftp["geometry"]
    crs = {'init': "epsg:4326"}
    gdf = GeoDataFrame(dftp, crs=crs, geometry=geometry)
    gdf.to_file("/home/mlopez/EXEC/GeoJsonMFFP/"+short+"_"+var+"_monthly.json", driver="GeoJSON")

In [ ]:
df45

### TODO:
- This dataframe should include both models (RCP 4.5 nad 8.5) and observations (historic)
- I should add labels and similar colors into the shiny graph

In [ ]:
df45.to_csv("p4tgmean.csv")

In [ ]:
plt.plot(years, df45["tg_mean_p50"], markersize='10', color='blue')
plt.fill_between(years, polysmooth(years, df45.tg_mean_p10,4), polysmooth(years, df45.tg_mean_p90,4), alpha=0.3, color='blue')

In [ ]:
dfmean2.to_csv("timeseries.csv")

### NOTE: this example was for the whole region. 
### TODO: 
- Split DF by regions and get same graph for each region
- Convert degrees in Celsius